In [5]:
import torch
import numpy as np
import cv2 as cv

from maskrcnn_benchmark.structures.bounding_box import BoxList
from maskrcnn_benchmark.structures.boxlist_ops import boxlist_iou

In [2]:
def cos_sim_matrix(
    a: torch.Tensor,
    b: torch.Tensor,
    eps: float = 1e-8
) -> torch.Tensor:
    """Computes a 2D cosine similarity matrix between two sets of vectors.

    Args:
        a (torch.Tensor): First list of vectors of shape (N, D).
        b (torch.Tensor): Second list of vectors of shape (M, D).
        eps (float, optional): Small value to avoid division by zero.
                               Defaults to 1e-8.

    Returns:
        torch.Tensor: A 2D cosine similarity matrix of shape (N, M).
        The cell at i-th row and j-th column represents the cosine similarity
        between the a[i] and b[j] D-dimensional vectors.
    """
    a_norm = a.norm(dim=1)[..., None]
    b_norm = b.norm(dim=1)[..., None]

    a_ = a / torch.clamp(a_norm, min=eps)
    b_ = b / torch.clamp(b_norm, min=eps)

    sim_matrix = torch.mm(a_, b_.T)

    return sim_matrix

def boxlist_feature_nms(
    boxes: BoxList,
    embs: torch.Tensor,
    iou_thresh_1: float = 0.1,
    iou_thresh_2: float = 0.9,
    cos_sim_thresh: float = 0.4,
    score_field: str = 'scores'
):
    assert len(boxes) == len(embs)
    assert embds.ndim == 2
    assert 0 < iou_thresh_1 < iou_thresh_2 < 1

    scores = boxes.get_field(score_field)
    boxes = boxes.convert('xyxy')
    iou_scores = boxlist_iou(boxes, boxes)
    cos_sim_scores = cos_sim_matrix(embs, embs)

    kept_indices = []

    for i in torch.argsort(scores).tolist():
        for j in kept_indices:
            iou = iou_scores[i, j]
            if iou > iou_thresh_2:
                break
            elif iou > iou_thresh_1:
                cos_sim = cos_sim_scores[i, j]
                if cos_sim > cos_sim_thresh:
                    break
        else:
            kept_indices.append(i)
    
    return kept_indices

SyntaxError: invalid syntax (Temp/ipykernel_7068/1885169875.py, line 1)

In [8]:
scores = torch.rand(10)
scores

tensor([0.4370, 0.4058, 0.5587, 0.0499, 0.0207, 0.7344, 0.3449, 0.7767, 0.8454,
        0.4787])

In [17]:
scores[[1, 2, 6]]

tensor([0.4058, 0.5587, 0.3449])